# NTRU Encrypt - PKE Scheme

O sistema criptográfico NTRU é um sistema de chave pública baseado em reticulados, tendo sido o primeiro deste tipo. 

## Parâmetros

Num sistema criptográfico NTRU, **f** (e **g**, se necessário) são as chaves privadas, enquanto h
é a chave pública. Essas chaves podem ser geradas através do seguinte algoritmo:

## Algoritmo de Geração de chaves

INPUT: Um conjunto de parâmetros $Param = \{N, p, q, d\}$ e uma $seed$.

1. Instanciar $Sampler$ com $\tau(d + 1, d)$ e com a $seed$;
2. $f ← Sampler$
3. se $f$ não é invertível $mod q$ então retornar ao passo 2
4. $g ← Sampler$
5. $h = g/(pf + 1) mod q$

OUTPUT: Chave pública $h$ e chave secreta $(pf, g)$

## Algoritmo de Cifragem

INPUT: Chave pública h, mensagem msg de comprimento mlen, um conjunto de parâmetros $Param$ e a $seed$.
1. $m = Pad(msg, seed)$
2. $rseed = Hash(m|h)$
3. Instanciar $Sampler$ com $T$ e com $rseed$;
4. $r ← Sampler$
5. $t = r ∗ h$
6. $tseed = Hash(t)$
7. Instanciar $Sampler$ com $T$ e com $tseed$;
8. $m_{mask} ← Sampler$
9. $m' = m − m_{mask}(mod p)$
10. $c = t + m$

OUTPUT: Criptograma **c**

O algoritmo acima usa um **método de *padding*** para lidar com
entropia insuficiente potencial de uma mensagem. Supondo que o tamanho da mensagem é válido e menor que $(N - 173)$ *bits*, o algoritmo *padding* funciona da seguinte maneira:

1. Converter $msg$ numa *string* de *bits*. Cada *bit* forma um coeficiente binário para a parte inferior do polinómio $m$, partindo do coeficiente 0.

2. Os últimos $167$ coeficientes de $m (x)$ são escolhidos aleatoriamente de $\{−1, 0, 1\}$ (com um *input* *seed* ). O que dá mais de $256$ bits de entropia.

3. O comprimento da msg é convertido numa *string* binária de 8 *bits* e forma os últimos $173$ a $168$ coeficientes de $m (x)$.

## Algorithm de Decifragem

INPUT: Chave secreta $f$, chave pública $h$, criptograma $c$, e o conjunto de parâmetros $Param$.
1. $m' = f ∗ c (mod p)$
2. $t = c − m$
3. $tseed = Hash(t)$
4. Instanciar $Sampler$ com $T$ e com $tseed$;
5. $m_{mask} = Sampler$
6. $m = m' + m_{mask} (mod p)$
7. $rseed = Hash(m|h)$
8. Instanciar $Sampler$ com $T$ e com $rseed$;
9. $r ← Sampler$
10. $msg, mlen = Extract(m)$
11. se $p · r ∗ h = t$ então
12. $result = msg, mlen$
13. caso contrário
14. $result = ⊥ $

OUTPUT: $result$

No algoritmo acima a operação $Extrair ()$ corresponde ao inverso de $Pad ()$. Emite uma mensagem $m$ e seu comprimento $mlen$.

In [4]:
from hashlib import sha512
from random import randint

CRYPTO_BYTES = 32

class NTRU(object):
    def __init__(self, manual, d, N=None, p=None, q=None):
        if manual:
            self.d = d
            self.N = N
            self.p = p
            self.q = q
        else:
            self.d = d
            self.N = next_prime(1 << self.d)
            self.p = 3
            self.q = next_prime(self.p*self.N)
        Z.<x>  = ZZ[]
        Q.<x>  = PolynomialRing(GF(self.q),name='x').quotient(x^self.N-1)
        self.Q = Q
        self.keygen()
        
    def keygen(self):
        # Instantiate Sampler with T (d + 1, d) and seed;
        seed =  int(sha512(bytes("12122121")).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, seed)
        F = self.Q(sampler.genTrinaryPolynomial())
        f = 1 + self.p*F
        while not f.is_unit():
            F = self.Q(sampler.genTrinaryPolynomial())
            f = 1 + self.p*F
        g = self.p*self.Q(sampler.genTrinaryPolynomial())
        h = self.rnd_modq(g/f)
        self.skey = (f,g)
        self.h = h

    def encrypt(self, msg, seed):
        padded_msg = self.pad_msg(msg, seed)

        rseed = int(sha512(bytes(padded_msg + self.h)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, rseed)
        r = self.Q(sampler.genTrinaryPolynomial())
        t = r*self.Q(self.h)

        tseed = int(sha512(bytes(t)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, tseed)
        msg_mask = self.Q(sampler.genTrinaryPolynomial())
        
        masked_msg = self.rnd_modp(self.rnd_modq(self.Q(padded_msg) - msg_mask))
        crypt = t + self.Q(masked_msg)
        return crypt.list()

    def decrypt(self, crypt):
        crypt = self.Q(crypt)
        masked_msg = self.rnd_modp(self.rnd_modq(self.skey[0] * crypt))
        t = crypt - self.Q(masked_msg)
        
        tseed = int(sha512(bytes(t)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, tseed)
        msg_mask = self.Q(sampler.genTrinaryPolynomial())
        padded_msg = self.rnd_modp(self.rnd_modq(self.Q(masked_msg) + msg_mask))

        rseed = int(sha512(bytes(padded_msg + self.h)).hexdigest(), 16)
        sampler = Sampler(self.N, self.d, self.d+1, rseed)
        r = self.Q(sampler.genTrinaryPolynomial())
        
        if r*self.Q(self.h) == t:
            msg, mlen = self.extract_msg(padded_msg)
            result = msg[:mlen]
        else:
            result = "Error"
        return result
    
    def rnd_modq(self, l):
        '''Round f mod q
        '''
        qq = (self.q-1)//2
        ll = map(lift,l.list())
        return [n if n <= qq else n - self.q  for n in ll]

    def rnd_modp(self, l):
        '''Round l mod p
        '''
        pp = (self.p-1)//2
        rr = lambda x: x if x <= pp else x - self.p
        return [rr(n%self.p) if n>=0 else -rr((-n)%self.p) for n in l]

    def pad_msg(self, msg, seed):
        ''' Pad message according to NTRU spec
        '''
        mlen = len(msg)
        if mlen < (self.N - 173):
            fill_len = self.N-173-mlen+1
            sampler = Sampler(167, self.d, self.d+1, seed)
            rand_pad = sampler.genTrinaryPolynomial()
            padded_msg = copy(msg)
            padded_msg.extend([0] * fill_len)
            padded_msg.extend(rand_pad)
            for i in range(5):
                padded_msg.append(mlen%2)
                mlen >>= 1
        return padded_msg
            
    def extract_msg(self, padded_msg):
        ''' Reverse padded message
        '''
        msg_len = 0
        for i in range(1,6):
            msg_len = msg_len*2+padded_msg[-i]
        return padded_msg, msg_len
    
    
class Sampler:
    def __init__(self, N, d1, d2, seed):
        self.N = N
        if d1 and d2:
            self.d1 = d1
            self.d2 = d2
        self.seed = seed
        
    def genBinaryPolynomial(self):
        f = [0]*self.N
        t = 0
        while t < self.d1:
            bit_count = ceil(log(self.N)) + 1
            k = self.seed%(2**bit_count)
            self.seed >>= bit_count
            i = k % self.N
            if f[i] == 0:
                f[i] = 1
                t += 1
        return f
    
    def genTrinaryPolynomial(self):
        f = self.genBinaryPolynomial()
        t = 0
        while t < self.d2:
            bit_count = ceil(log(self.N)) + 1
            k = self.seed%(2**bit_count)
            self.seed >>= bit_count
            i = k % self.N
            if f[i] == 0:
                f[i] = -1
                t += 1
        return f  

In [5]:
K = NTRU(False, 10)
#K = NTRU(True, 10, N=443, q=2048, p=3) #NTRU-443
msg = [choice([-1,0,1]) for i in range(10)]
seed =  int(sha512(bytes("12122121")).hexdigest(), 16)
e = K.encrypt(msg, seed)
d = K.decrypt(e)
print(d==msg)

True
